## Import Dataset and EDA

In [1]:
#Modules
import pandas as pd
import numpy as np
import pickle
import re

compare your dataset with Kremlin to see what url actually overlap. they cannot overlap cause url from kremlin are all from http://kremlin.ru/events/president/news

## data clean

### messages to the federal assembly

In [ ]:
#messages to the federal assembly (mtfa)
mtfa = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/messages_to_federal_assembly.pkl')
mtfa.info()

In [ ]:
#detect the fist part before the actual speech
for i in range(14):
    text = mtfa['text'][i]
    match = re.search(r': Уважаем|: Добрый', text)
    if match:
        text_before = text[:match.start()]
        print(f"Row {i}: {text_before}")
        print(f"Number of characters: {len(text_before)}")
        print(f"--------------------------")
    else:
        print(f"Row {i}: No match found.")

In [ ]:
#detect the last part after the actual speech
for i in range(len(mtfa)):
    text = mtfa['text'][i]
    match = re.search(r'Опубликован.*$', text)
    if match:
        text_after = match.group(0)
        print(f"Row {i}: {text_after}")
        print(f"Number of characters: {len(text_after)}")
        print(f"--------------------------")
    else:
        print(f"Row {i}: No match found.")


In [ ]:
# alternative way to detect the last words
for i in range(14):
    text = mtfa['text'][i]
    last_33_words = ' '.join(text.split()[-33:])
    print(f"Row {i}: {last_33_words}")

In [ ]:
def clean(text):
    # replace actual newline characters \n with a space
    text = re.sub(r'\n', ' ', text)
    #replace last part with a space
    match = re.search(r'Опубликован.*$', text)
    if match:
        text_after = match.group(0)
        text = text.replace(text_after, ' ')
    #replace initial part with a white space
    match = re.search(r'Уважаем|Добрый', text)
    if match:
        text = ' ' + text[match.start():]
    
    return text.strip()


In [ ]:
mtfa['text_clean'] = mtfa['text'].map(clean)
mtfa['text_clean']

In [ ]:
# after applying the transformation
for i in range(14):
    text = mtfa['text_clean'][i]
    last_33_words = ' '.join(text.split()[-33:])
    print(f"Row {i}: {last_33_words}")

In [ ]:
# Before cleaning
for i in range(len(mtfa)):
    text = mtfa['text'][i]
    num_words = len(text.split())
    print(f"Row {i}: Number of words before cleaning: {num_words}")
print('-----------------------')
# After cleaning
for i in range(len(mtfa)):
    text = mtfa['text_clean'][i]
    num_words = len(text.split())
    print(f"Row {i}: Number of words after cleaning: {num_words}")


identify and remove noise with regex

In [ ]:
mtfa['text_clean'].iloc[0]

In [ ]:
#Identify Noise with Regular Expression
import re 
RE_SUSPICIOUS = re.compile(r'[xa0#<>{}\[\]\\]')

def impurity(text, min_len=10): 
    """returns the share of suspicious characters in a text""" 
    if text == None or len(text) < min_len: 
        return 0 
    else: 
        return len(RE_SUSPICIOUS.findall(text))/len(text)


In [ ]:
mtfa['impurity'] = mtfa['text_clean'].apply(impurity)

In [ ]:
mtfa['impurity'].sort_values(ascending=False).value_counts()

In [ ]:
# get the top 3 records 
#critical part is just []...I will leave as it is for the moment
mtfa[['text_clean', 'impurity']].sort_values(by='impurity', ascending=False).head(3)

In [ ]:
def clean_2(text):
    # replace everything non-alphanumeric with a space
    text = re.sub(r'\W+', ' ', text)
    # replace two or more dots with one
    text = re.sub(r'\.{2,}', ' ', text)
    # replace sequences of white spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # convert text to lowercase
    text = text.lower()
    # replace \xa0 with a white space
    text = re.sub(r'\xa0', ' ', text)
    return text.strip()

In [ ]:
mtfa['text_clean'] = mtfa['text_clean'].map(clean_2)
mtfa['text_clean']

In [ ]:
mtfa['text_clean'].iloc[0]

In [ ]:
mtfa.drop(columns=['impurity'], inplace=True)

In [ ]:
mtfa['speaker'] = mtfa['speaker'].replace({'* * *, В.Путин:': 'В.Путин:', 'Д.Медведев:, * * *': 'Д.Медведев:'})

In [ ]:
#save the dataset (overwrite the previous one)
mtfa.to_pickle('data/corpus_adriana/corpus_president_of_russia/messages_to_federal_assembly.pkl')

In [61]:
mtfa = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/messages_to_federal_assembly.pkl')
mtfa

,date,URL,title,speaker,text,text_clean
0,2020-01-15,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,уважаемые члены совета федерации и депутаты го...
1,2015-12-03,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,уважаемые члены совета федерации уважаемые деп...
2,2013-12-12,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,уважаемые граждане россии уважаемые члены сове...
3,2012-12-12,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,уважаемые граждане россии уважаемые члены сове...
4,2016-12-01,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,добрый день уважаемые коллеги уважаемые члены ...
5,2018-03-01,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,уважаемые граждане россии уважаемые члены сове...
6,2009-11-12,http://kremlin.ru/events/president/transcripts...,Послание Федеральному Собранию Российской Феде...,Д.Медведев:,Все материалы сайта доступны по лицензии: Crea...,уважаемые граждане россии уважаемые депутаты и...
7,2021-04-21,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,уважаемые члены совета федерации депутаты госу...
8,2014-12-04,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,уважаемые члены совета федерации депутаты госу...
9,2019-02-20,http://kremlin.ru/events/president/transcripts...,Послание Президента Федеральному Собранию,В.Путин:,Все материалы сайта доступны по лицензии: Crea...,уважаемые члены совета федерации уважаемые деп...


In [62]:
for i in range(len(mtfa)):
    text = mtfa['text_clean'][i]
    num_words = len(text.split())
    print(f"Row {i}: Number of words after cleaning: {num_words}")

Row 0: Number of words after cleaning: 7575
Row 1: Number of words after cleaning: 6577
Row 2: Number of words after cleaning: 7959
Row 3: Number of words after cleaning: 9726
Row 4: Number of words after cleaning: 7766
Row 5: Number of words after cleaning: 12812
Row 6: Number of words after cleaning: 9891
Row 7: Number of words after cleaning: 7949
Row 8: Number of words after cleaning: 7074
Row 9: Number of words after cleaning: 9876
Row 10: Number of words after cleaning: 8493
Row 11: Number of words after cleaning: 10609
Row 12: Number of words after cleaning: 6720
Row 13: Number of words after cleaning: 7320


### statements on major issues

In [2]:
# statements on major issues (somi)
somi = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/statements_on_major_issues.pkl')
somi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67656 entries, 0 to 67655
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     67656 non-null  object
 1   url      67656 non-null  object
 2   title    67656 non-null  object
 3   speaker  61807 non-null  object
 4   text     67656 non-null  object
dtypes: object(5)
memory usage: 2.6+ MB


In [3]:
somi['speaker'] = somi['speaker'].replace({'В.Путин': 'В.Путин:', 'Putin:':'В.Путин:', 'Д.Медведев': 'Д.Медведев:'})

In [4]:
#speakers 
somi.speaker.value_counts()

В.Путин:                                            24971
Д.Медведев:                                          9511
Вопрос:                                               914
Д.Песков:                                             364
Ф.Лукьянов:                                           341
                                                    ...  
А.Чачава:                                               1
Х.-В.Штейнфельд:                                        1
Из зала:                                                1
В.Мельниченко                                           1
Стенографический\nотчёт о заседании РОК «Победа»        1
Name: speaker, Length: 1756, dtype: int64

In [5]:
somi.speaker.isna().value_counts()

False    61807
True      5849
Name: speaker, dtype: int64

In [6]:
somi.loc[(somi['speaker'].isna()) & (somi['text'].str.contains('Д.Медведев:')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['text'].str.contains('В.Путин')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['text'].str.contains('Вопрос:')), 'speaker'] = 'Вопрос:'

In [7]:
#for url with only one speaker medvedev
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/1')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/2088')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/976')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/8596')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/787')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/7685')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/634')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/633')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/5989')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/591')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/5566')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/5552')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/2283')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/859')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/2445')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/2592')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/3257')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/3387')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/3805')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/3921')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/4186')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/4336')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/4358')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/5209')), 'speaker'] = 'Д.Медведев:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/5269')), 'speaker'] = 'Д.Медведев:'


In [8]:
#for url with only one speaker putin
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/2$')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22147')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22197')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22204')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22237')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22279')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22281')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22338')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22384')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22394')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22395')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22401')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22442')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/63527')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22403')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22545')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22546')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22549')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22714')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22777')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22838')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22949')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22979')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23009')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23066')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23619')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/63888')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23547')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23656')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23791')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23820')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23857')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/2393')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/24642')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/24809')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/24864')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22619')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23407')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23437')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23503')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/23512')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/49957')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/54467')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/63560')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/64270')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/67551')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/68366')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/47045')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/69299')), 'speaker'] = 'В.Путин:'
somi.loc[(somi['speaker'].isna()) & (somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/67044')), 'speaker'] = 'В.Путин:'

In [9]:
somi.speaker.isna().value_counts()

False    65401
True      2255
Name: speaker, dtype: int64

In [10]:
condition_1 = somi[somi['speaker'].isna()]['text'].str.contains('Все материалы')
condition_2 = somi[somi['speaker'].isna()]['text'].str.contains('\* \* \*')
condition_3 = somi[somi['speaker'].isna()]['text'].str.strip().str.len() == 0
condition_4 = somi[somi['speaker'].isna()]['text'].str.contains('Глава государства|Глава|Президент России|Президент')
condition_5 = somi[somi['speaker'].isna()]['text'].str.contains('http://')
condition_6 = somi[somi['speaker'].isna()]['text'].str.contains('Дата публикации')
condition_7 = somi[somi['speaker'].isna()]['text'].str.contains('Ссылка на материал:')
condition_8 = somi[somi['speaker'].isna()]['text'].str.contains('Опубликован в разделе|Опубликован в разделах|Официальный сайт')
condition_9 = somi[somi['speaker'].isna()]['text'].str.contains('Владимир Путин|Дмитрий Медведев')
condition_10 = somi[somi['speaker'].isna()]['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/6769|http://kremlin.ru/events/president/transcripts/statements/6074')

filtered_somi = somi[~(somi['speaker'].isna() & 
                       (condition_1 | condition_2 | condition_3| condition_4| condition_5| condition_6| condition_7|condition_8|condition_9|condition_10))]

In [11]:
filtered_somi.speaker.isna().value_counts()

False    65401
True       665
Name: speaker, dtype: int64

In [12]:
url_pattern = 'http://kremlin.ru/events/president/transcripts/statements/2134'
filtered_somi.loc[(filtered_somi['speaker'].isna()) & (filtered_somi['url'].str.contains(url_pattern))]

,date,url,title,speaker,text
47372,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,Главная тема съезда – долгосрочная стратегия р...
47373,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,На форуме собрались более шестисот делегатов и...
47375,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,Рад приветствовать делегатов и гостей съезда. ...
47376,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,Прежде всего мне бы хотелось поблагодарить вас...
47377,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,Благодаря парламентскому влиянию вашей партии ...
...,...,...,...,...,...
47539,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,Партия «Единая Россия» отстаивала и будет отст...
47540,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,Мы с вами будем это делать и сделаем.
47541,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,Благодарю за внимание.
47542,2008-11-20,http://kremlin.ru/events/president/transcripts...,Х съезд партии «Единая Россия»,None,"Опубликован в разделах: ,"


In [13]:
#the breakpoint
break_point = filtered_somi.loc[(filtered_somi['speaker'].isna()) & (filtered_somi['url'].str.contains(url_pattern)) & (filtered_somi['text'].str.contains('<…>'))]

In [14]:
# loop over the rows that match the filter criteria
for idx, row in filtered_somi.loc[(filtered_somi['speaker'].isna()) & (filtered_somi['url'].str.contains(url_pattern))].iterrows():
    if str(idx) < str(break_point):
        filtered_somi.at[idx, 'speaker'] = 'Medvedev:'
    else:
        filtered_somi.at[idx, 'speaker'] = 'Putin:'


In [15]:
filtered_somi.speaker.isna().value_counts()

False    65568
True       498
Name: speaker, dtype: int64

In [16]:
filtered_somi.loc[(filtered_somi['speaker'].isna()) & (filtered_somi['text'].str.contains('Г.Шрёдер:')), 'speaker'] = 'Г.Шрёдер:'

/var/folders/mm/gw92vpys6rggjz_4z42s244w0000gn/T/ipykernel_4286/3325061389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_somi.loc[(filtered_somi['speaker'].isna()) & (filtered_somi['text'].str.contains('Г.Шрёдер:')), 'speaker'] = 'Г.Шрёдер:'


In [17]:
row_to_fill = filtered_somi.loc[filtered_somi['url'].str.contains('http://kremlin.ru/events/president/transcripts/statements/22950')].copy()
row_to_fill['speaker'].fillna(method='ffill', inplace=True)
filtered_somi.loc[row_to_fill.index] = row_to_fill

/var/folders/mm/gw92vpys6rggjz_4z42s244w0000gn/T/ipykernel_4286/1007969576.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_somi.loc[row_to_fill.index] = row_to_fill


In [18]:
filtered_somi.speaker.isna().value_counts()

False    65623
True       443
Name: speaker, dtype: int64

In [19]:
filtered_somi.drop(filtered_somi.loc[filtered_somi['speaker'].isna() & (filtered_somi['date'] < '2014-01-01')].index, inplace=True)

/var/folders/mm/gw92vpys6rggjz_4z42s244w0000gn/T/ipykernel_4286/84537534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_somi.drop(filtered_somi.loc[filtered_somi['speaker'].isna() & (filtered_somi['date'] < '2014-01-01')].index, inplace=True)


In [20]:
filtered_somi.speaker.isna().value_counts()

False    65623
True       271
Name: speaker, dtype: int64

In [21]:
filtered_somi.drop(filtered_somi.loc[filtered_somi['speaker'].isna()].index, inplace=True)

/var/folders/mm/gw92vpys6rggjz_4z42s244w0000gn/T/ipykernel_4286/2892791293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_somi.drop(filtered_somi.loc[filtered_somi['speaker'].isna()].index, inplace=True)


In [22]:
filtered_somi.reset_index(drop=True)

,date,url,title,speaker,text
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,А.Бортников:,"Дмитрий Анатольевич, сегодня в 7.57 произошел ..."
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,А.Бортников:,По предварительной взрывотехнической экспертиз...
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,А.Бортников:,"По нашей предварительной версии, эти террорист..."
...,...,...,...,...,...
65618,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,
65619,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Официальный сайт президента России:
65620,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Инвестиционный форум «Россия зовёт!»
65621,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,http://kremlin.ru/events/president/transcripts...


In [23]:
speaker_counts = filtered_somi.speaker.value_counts()

In [24]:
# Filter the speaker counts to keep only those with a count of 200 or greater, excluding 'Вопрос:' and '* * *'
filtered_speaker_counts = speaker_counts[(speaker_counts >= 200) & (speaker_counts.index != 'Вопрос:') & (speaker_counts.index != '* * *')]
# Create a list of the speakers to keep
speakers_to_keep = list(filtered_speaker_counts.index)
# Filter the original DataFrame to keep only rows with the desired speakers. assign a new df
somi_filtered_speaker = filtered_somi[filtered_somi['speaker'].isin(speakers_to_keep)]

In [26]:
somi_filtered_speaker.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41023 entries, 8 to 67655
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     41023 non-null  object
 1   url      41023 non-null  object
 2   title    41023 non-null  object
 3   speaker  41023 non-null  object
 4   text     41023 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [27]:
somi_filtered_speaker

,date,url,title,speaker,text
8,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...
9,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...
14,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,", что сделали спасатели, какова сейчас ситуаци..."
15,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,"Дмитрий Анатольевич, все системы сработали дос..."
16,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,Задействованы были все необходимые силы: по ли...
...,...,...,...,...,...
67651,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,
67652,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Официальный сайт президента России:
67653,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Инвестиционный форум «Россия зовёт!»
67654,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,http://kremlin.ru/events/president/transcripts...


In [2]:
#assign this df before continuing the cleaning
somi_filtered_speaker.to_pickle('data/corpus_adriana/corpus_president_of_russia/statements_on_major_issues.pkl')

NameError: name 'somi_filtered_speaker' is not defined

##### continue the data cleaning of somi corpus

In [24]:
#read the df
somi = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/statements_on_major_issues.pkl')
somi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41023 entries, 8 to 67655
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     41023 non-null  object
 1   url      41023 non-null  object
 2   title    41023 non-null  object
 3   speaker  41023 non-null  object
 4   text     41023 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [27]:
somi.reset_index(drop=True, inplace=True)

In [28]:
somi

,date,url,title,speaker,text
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,", что сделали спасатели, какова сейчас ситуаци..."
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,"Дмитрий Анатольевич, все системы сработали дос..."
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,Задействованы были все необходимые силы: по ли...
...,...,...,...,...,...
41018,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,
41019,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Официальный сайт президента России:
41020,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Инвестиционный форум «Россия зовёт!»
41021,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,http://kremlin.ru/events/president/transcripts...


In [29]:
somi['text'].str.contains('Путин:|:Медведев').value_counts()

False    40944
True        79
Name: text, dtype: int64

In [30]:
filtered_text = somi[somi['text'].str.contains('Ссылка на материал:')]
word_counts = filtered_text['text'].str.split().str.len()
word_counts

26       3
131      3
166      3
195      3
722      3
        ..
40675    3
40770    3
40794    3
40842    3
41016    3
Name: text, Length: 469, dtype: int64

In [31]:
somi[somi['text'].str.contains('Официальный сайт')]

,date,url,title,speaker,text
29,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Официальный сайт президента России:
134,2016-01-12,http://kremlin.ru/events/president/transcripts...,Интервью немецкому изданию Bild. Часть 2,Реплика:,Официальный сайт президента России:
169,2009-06-06,http://kremlin.ru/events/president/transcripts...,Выступление на заседании Всемирного зернового ...,Д.Медведев:,Официальный сайт президента России:
198,2008-09-11,http://kremlin.ru/events/president/transcripts...,Вступительное слово на совещании по вопросам с...,Д.Медведев:,Официальный сайт президента России:
725,2004-01-27,http://kremlin.ru/events/president/transcripts...,Выступление на заседании президиума Государств...,В.Путин:,Официальный сайт президента России:
...,...,...,...,...,...
40678,2010-03-29,http://kremlin.ru/events/president/transcripts...,Дмитрий Медведев почтил память жертв взрывов в...,Д.Медведев:,Официальный сайт президента России:
40773,2003-12-03,http://kremlin.ru/events/president/transcripts...,Вступительное слово на заседании Совета Безопа...,В.Путин:,Официальный сайт президента России:
40797,2016-10-05,http://kremlin.ru/events/president/transcripts...,Встреча с сотрудниками Службы внешней разведки,В.Путин:,Официальный сайт президента России:
40845,2014-03-21,http://kremlin.ru/events/president/transcripts...,Расширенное заседание коллегии МВД,В.Путин:,Официальный сайт президента России:


In [32]:
condition_1 = somi['text'].str.contains('Все материалы')
condition_2 = somi['text'].str.strip().str.len() == 0
condition_3 = somi['text'].str.contains('\* \* \*')
condition_4 = somi['text'].str.contains('http://')
condition_5 = somi['text'].str.contains('Дата публикации')
condition_6 = somi['text'].str.contains('Ссылка на материал:')
condition_7 = somi['text'].str.contains('Официальный сайт')
filtered_somi = somi[~((condition_1 | condition_2 | condition_3| condition_4| condition_5| condition_6| condition_7))]

In [33]:
filtered_somi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37589 entries, 0 to 41022
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     37589 non-null  object
 1   url      37589 non-null  object
 2   title    37589 non-null  object
 3   speaker  37589 non-null  object
 4   text     37589 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [34]:
filtered_somi.reset_index(drop=True, inplace=True)

In [35]:
filtered_somi

,date,url,title,speaker,text
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,", что сделали спасатели, какова сейчас ситуаци..."
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,"Дмитрий Анатольевич, все системы сработали дос..."
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,Задействованы были все необходимые силы: по ли...
...,...,...,...,...,...
37584,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо\nВам большое.
37585,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо вам большое.
37586,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,"Опубликован в разделах: ,"
37587,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Инвестиционный форум «Россия зовёт!»


In [ ]:
def clean(text):
    # replace actual newline characters \n with a space
    text = re.sub(r'\n', ' ', text)
    # replace everything non-alphanumeric with a space
    text = re.sub(r'\W+', ' ', text)
    # replace two or more dots with one
    text = re.sub(r'\.{2,}', ' ', text)
    # replace sequences of white spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # convert text to lowercase
    text = text.lower()
    # replace \xa0 with a white space
    text = re.sub(r'\xa0', ' ', text)
    #replace Д.Медведев:|В.Путин: with a white space
    text = re.sub(r'(Д\.Медведев:|В\.Путин:)', ' ', text)
    return text.strip()

In [ ]:
filtered_somi_copy = filtered_somi.copy()
filtered_somi_copy.loc[:, 'text_clean'] = filtered_somi['text'].map(clean)

In [ ]:
filtered_somi_copy

,date,url,title,speaker,text,text_clean
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...,сегодня в московском метро произошли террорист...
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...,теперь по сути происшедшего первый александр в...
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,", что сделали спасатели, какова сейчас ситуаци...",что сделали спасатели какова сейчас ситуация э...
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,"Дмитрий Анатольевич, все системы сработали дос...",дмитрий анатольевич все системы сработали дост...
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,Задействованы были все необходимые силы: по ли...,задействованы были все необходимые силы по лин...
...,...,...,...,...,...,...
37582,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,"В отношении инфраструктуры, Вы знаете, у нас е...",в отношении инфраструктуры вы знаете у нас ест...
37583,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,А Вам\nя желаю всего самого доброго – и в Ваше...,а вам я желаю всего самого доброго и в вашей п...
37584,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо\nВам большое.,спасибо вам большое
37585,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо вам большое.,спасибо вам большое


In [51]:
filtered_somi_copy[filtered_somi_copy['text_clean'].str.contains('администрация президента россии')]

,date,url,title,speaker,text,text_clean
25,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Администрация Президента России 2023 год,администрация президента россии 2023 год
122,2016-01-12,http://kremlin.ru/events/president/transcripts...,Интервью немецкому изданию Bild. Часть 2,Реплика:,Администрация Президента России 2023 год,администрация президента россии 2023 год
149,2009-06-06,http://kremlin.ru/events/president/transcripts...,Выступление на заседании Всемирного зернового ...,Д.Медведев:,Администрация Президента России 2023 год,администрация президента россии 2023 год
170,2008-09-11,http://kremlin.ru/events/president/transcripts...,Вступительное слово на совещании по вопросам с...,Д.Медведев:,Администрация Президента России 2023 год,администрация президента россии 2023 год
689,2004-01-27,http://kremlin.ru/events/president/transcripts...,Выступление на заседании президиума Государств...,В.Путин:,Администрация Президента России 2023 год,администрация президента россии 2023 год
...,...,...,...,...,...,...
37273,2010-03-29,http://kremlin.ru/events/president/transcripts...,Дмитрий Медведев почтил память жертв взрывов в...,Д.Медведев:,Администрация Президента России 2023 год,администрация президента россии 2023 год
37360,2003-12-03,http://kremlin.ru/events/president/transcripts...,Вступительное слово на заседании Совета Безопа...,В.Путин:,Администрация Президента России 2023 год,администрация президента россии 2023 год
37378,2016-10-05,http://kremlin.ru/events/president/transcripts...,Встреча с сотрудниками Службы внешней разведки,В.Путин:,Администрация Президента России 2023 год,администрация президента россии 2023 год
37420,2014-03-21,http://kremlin.ru/events/president/transcripts...,Расширенное заседание коллегии МВД,В.Путин:,Администрация Президента России 2023 год,администрация президента россии 2023 год


In [52]:
filtered_somi_copy[filtered_somi_copy['text_clean'].str.contains('опубликован в разделах')]

,date,url,title,speaker,text,text_clean
23,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,"Опубликован в разделах: , , ,",опубликован в разделах
120,2016-01-12,http://kremlin.ru/events/president/transcripts...,Интервью немецкому изданию Bild. Часть 2,Реплика:,"Опубликован в разделах: ,",опубликован в разделах
732,2019-07-09,http://kremlin.ru/events/president/transcripts...,II Глобальный саммит по производству и индустр...,В.Путин:,"Опубликован в разделах: ,",опубликован в разделах
830,2011-11-21,http://kremlin.ru/events/president/transcripts...,Встреча с офицерами Южного военного округа,Д.Медведев:,"Опубликован в разделах: ,",опубликован в разделах
915,2018-07-19,http://kremlin.ru/events/president/transcripts...,Совещание послов и постоянных представителей Р...,В.Путин:,"Опубликован в разделах: ,",опубликован в разделах
...,...,...,...,...,...,...
37259,2022-09-21,http://kremlin.ru/events/president/transcripts...,Обращение Президента Российской Федерации,В.Путин:,"Опубликован в разделах: ,",опубликован в разделах
37271,2010-03-29,http://kremlin.ru/events/president/transcripts...,Дмитрий Медведев почтил память жертв взрывов в...,Д.Медведев:,"Опубликован в разделах: ,",опубликован в разделах
37376,2016-10-05,http://kremlin.ru/events/president/transcripts...,Встреча с сотрудниками Службы внешней разведки,В.Путин:,"Опубликован в разделах: ,",опубликован в разделах
37418,2014-03-21,http://kremlin.ru/events/president/transcripts...,Расширенное заседание коллегии МВД,В.Путин:,"Опубликован в разделах: ,",опубликован в разделах


In [53]:
condition_1 = filtered_somi_copy['text_clean'].str.contains('опубликован в разделах')
condition_2 = filtered_somi_copy['text_clean'].str.contains('администрация президента россии')
filtered_somi = filtered_somi_copy[~((condition_1 | condition_2))]

In [55]:
filtered_somi

,date,url,title,speaker,text,text_clean
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...,сегодня в московском метро произошли террорист...
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...,теперь по сути происшедшего первый александр в...
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,", что сделали спасатели, какова сейчас ситуаци...",что сделали спасатели какова сейчас ситуация э...
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,"Дмитрий Анатольевич, все системы сработали дос...",дмитрий анатольевич все системы сработали дост...
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,Задействованы были все необходимые силы: по ли...,задействованы были все необходимые силы по лин...
...,...,...,...,...,...,...
37582,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,"В отношении инфраструктуры, Вы знаете, у нас е...",в отношении инфраструктуры вы знаете у нас ест...
37583,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,А Вам\nя желаю всего самого доброго – и в Ваше...,а вам я желаю всего самого доброго и в вашей п...
37584,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо\nВам большое.,спасибо вам большое
37585,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо вам большое.,спасибо вам большое


In [2]:
somi = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/statements_on_major_issues.pkl')
somi

,date,url,title,speaker,text,text_clean
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...,сегодня в московском метро произошли террорист...
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...,теперь по сути происшедшего первый александр в...
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,", что сделали спасатели, какова сейчас ситуаци...",что сделали спасатели какова сейчас ситуация э...
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,"Дмитрий Анатольевич, все системы сработали дос...",дмитрий анатольевич все системы сработали дост...
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,Задействованы были все необходимые силы: по ли...,задействованы были все необходимые силы по лин...
...,...,...,...,...,...,...
37582,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,"В отношении инфраструктуры, Вы знаете, у нас е...",в отношении инфраструктуры вы знаете у нас ест...
37583,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,А Вам\nя желаю всего самого доброго – и в Ваше...,а вам я желаю всего самого доброго и в вашей п...
37584,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо\nВам большое.,спасибо вам большое
37585,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо вам большое.,спасибо вам большое


In [34]:
def clean(text):
    #replace Д.Медведев:|В.Путин: with a white space
    text = re.sub(r'(Д\.Медведев:|В\.Путин:)', ' ', text)
    # replace actual newline characters \n with a space
    text = re.sub(r'\n', ' ', text)
    # replace everything non-alphanumeric with a space
    text = re.sub(r'\W+', ' ', text)
    # replace two or more dots with one
    text = re.sub(r'\.{2,}', ' ', text)
    # replace sequences of white spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # convert text to lowercase
    text = text.lower()
    # replace \xa0 with a white space
    text = re.sub(r'\xa0', ' ', text)
    return text.strip()

In [35]:
somi_copy = somi.copy()
somi_copy.loc[:, 'text_clean'] = somi['text'].map(clean)

In [36]:
somi_copy

,date,url,title,speaker,text,text_clean
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...,сегодня в московском метро произошли террорист...
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...,теперь по сути происшедшего первый александр в...
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,", что сделали спасатели, какова сейчас ситуаци...",что сделали спасатели какова сейчас ситуация э...
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,"Дмитрий Анатольевич, все системы сработали дос...",дмитрий анатольевич все системы сработали дост...
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,Задействованы были все необходимые силы: по ли...,задействованы были все необходимые силы по лин...
...,...,...,...,...,...,...
37582,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,"В отношении инфраструктуры, Вы знаете, у нас е...",в отношении инфраструктуры вы знаете у нас ест...
37583,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,А Вам\nя желаю всего самого доброго – и в Ваше...,а вам я желаю всего самого доброго и в вашей п...
37584,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо\nВам большое.,спасибо вам большое
37585,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо вам большое.,спасибо вам большое


In [37]:
word_counts = somi_copy['text'].str.split().str.len()
word_counts

0        14
1        20
2        14
3        54
4        71
         ..
37582    56
37583    17
37584     3
37585     3
37587     4
Name: text, Length: 36766, dtype: int64

In [38]:
somi_copy[somi_copy['text_clean'].str.contains('медведев|путин')]

,date,url,title,speaker,text,text_clean
26,2012-03-23,http://kremlin.ru/events/president/transcripts...,Конференция Российского совета по международны...,Д.Медведев:,Дмитрий Медведев принял участие в научно-практ...,дмитрий медведев принял участие в научно практ...
28,2012-03-23,http://kremlin.ru/events/president/transcripts...,Конференция Российского совета по международны...,Д.Медведев:,На полях конференции Дмитрий Медведев встретил...,на полях конференции дмитрий медведев встретил...
173,2019-06-20,http://kremlin.ru/events/president/transcripts...,Прямая линия с Владимиром Путиным,Т.Ремезова:,Почти\n400 тысяч СМС- и ММС-сообщений. Но в эт...,почти 400 тысяч смс и ммс сообщений но в этом ...
735,2011-11-21,http://kremlin.ru/events/president/transcripts...,Встреча с офицерами Южного военного округа,Д.Медведев:,В ходе поездки в Северную Осетию – Аланию Дмит...,в ходе поездки в северную осетию аланию дмитри...
921,2020-05-09,http://kremlin.ru/events/president/transcripts...,75-летие Победы,Д.Медведев:,По окончании церемонии с Ивановской площади Кр...,по окончании церемонии с ивановской площади кр...
...,...,...,...,...,...,...
36934,2011-08-05,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева,Д.Медведев:,"Саркози сделал очень добрую вещь, он позвонил ...",саркози сделал очень добрую вещь он позвонил м...
36937,2011-08-05,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева,Д.Медведев:,Что касается плана Медведева – Саркози. Он воо...,что касается плана медведева саркози он вообще...
36938,2011-08-05,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева,Д.Медведев:,"Уважаемые друзья, могу вам сказать одну вещь. ...",уважаемые друзья могу вам сказать одну вещь ес...
36986,2011-08-05,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева,Д.Медведев:,Интервью Дмитрия Медведева,интервью дмитрия медведева


In [42]:
description = somi_copy[somi_copy['text_clean'].str.contains('дмитрий медведев|владимир путин|пресидент') & 
                        ~somi_copy['text_clean'].str.contains('уважаем|добр|дорог')]

In [44]:
description.reset_index(inplace=True, drop=True)

In [45]:
word_counts = description['text'].str.split().str.len()
word_counts

0      18
1      12
2      26
3      17
4      10
       ..
103    28
104     7
105    10
106    14
107     9
Name: text, Length: 108, dtype: int64

In [46]:
somi_copy.drop(somi_copy[somi_copy['text_clean'].str.contains('дмитрий медведев|владимир путин|пресидент') & 
                          ~somi_copy['text_clean'].str.contains('уважаем|добр|дорог')].index, inplace=True)

In [47]:
somi_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36658 entries, 0 to 37587
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        36658 non-null  object
 1   url         36658 non-null  object
 2   title       36658 non-null  object
 3   speaker     36658 non-null  object
 4   text        36658 non-null  object
 5   text_clean  36658 non-null  object
dtypes: object(6)
memory usage: 2.0+ MB


In [56]:
somi_copy.to_pickle('data/corpus_adriana/corpus_president_of_russia/statements_on_major_issues.pkl')

#### somi concatenated

In [46]:
somi = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/statements_on_major_issues.pkl')
somi

,date,url,title,speaker,text,text_clean
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,: Сегодня в московском метро произошли террори...,сегодня в московском метро произошли террорист...
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,Теперь по сути происшедшего. Первый – Александ...,теперь по сути происшедшего первый александр в...
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,Д.Медведев:,", что сделали спасатели, какова сейчас ситуаци...",что сделали спасатели какова сейчас ситуация э...
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,"Дмитрий Анатольевич, все системы сработали дос...",дмитрий анатольевич все системы сработали дост...
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,С.Шойгу:,Задействованы были все необходимые силы: по ли...,задействованы были все необходимые силы по лин...
...,...,...,...,...,...,...
37582,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,"В отношении инфраструктуры, Вы знаете, у нас е...",в отношении инфраструктуры вы знаете у нас ест...
37583,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,А Вам\nя желаю всего самого доброго – и в Ваше...,а вам я желаю всего самого доброго и в вашей п...
37584,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо\nВам большое.,спасибо вам большое
37585,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Спасибо вам большое.,спасибо вам большое


In [47]:
def concat_strings(x):
    return ' '.join(x)

In [48]:
somi_concatenated = somi.groupby(['date', 'url', 'title', 'speaker'], as_index=False).agg({'text': concat_strings})

In [49]:
somi_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691 entries, 0 to 690
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     691 non-null    object
 1   url      691 non-null    object
 2   title    691 non-null    object
 3   speaker  691 non-null    object
 4   text     691 non-null    object
dtypes: object(5)
memory usage: 27.1+ KB


In [51]:
somi_concatenated.text.iloc[0]

'В.Путин: Добрый день, уважаемые коллеги! Сегодня я подписал закон о\xa0местном самоуправлении, над которым и\xa0Администрация, и\xa0Правительство, и\xa0депутаты Государственной Думы очень долго работали. Этот документ\xa0– одна из\xa0важнейших составляющих полномасштабной административной реформы, о\xa0которой мы много говорим. Я знаю, что на\xa0этой неделе на\xa0Правительстве рассматриваются вопросы в\xa0этом контексте. Два слова об\xa0этом законе. Местное самоуправление, мы хорошо знаем, это дополнительные гарантии прав граждан в\xa0управлении делами государства. Это приближение власти непосредственно к\xa0людям. Повышение социальной активности граждан. Это также предотвращение чрезмерной концентрации власти в\xa0центре. Дополнительные гарантии устойчивости политической системы. Закон вступает в\xa0силу с\xa01\xa0января 2006\xa0года. Мы предусмотрели очень длительный переходный период для того, чтобы как следует подготовиться к\xa0его реализации. Сделано это не\xa0случайно. Сам зако

In [55]:
def clean(text):
    # replace actual newline characters \n with a space
    text = re.sub(r'\n', ' ', text)
    #replace Д.Медведев:|В.Путин: with a white space
    text = re.sub(r'(Д\.Медведев:|В\.Путин:)', ' ', text)
    # replace everything non-alphanumeric with a space
    text = re.sub(r'\W+', ' ', text)
    # replace two or more dots with one
    text = re.sub(r'\.{2,}', ' ', text)
    # replace sequences of white spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # convert text to lowercase
    text = text.lower()
    # replace \xa0 with a white space
    text = re.sub(r'\xa0', ' ', text)
    return text.strip()

In [56]:
somi_concatenated['text_clean'] = somi_concatenated['text'].map(clean)

In [58]:
somi_concatenated.to_pickle('data/corpus_adriana/corpus_president_of_russia/somi_concatenated.pkl')

### interviews

In [2]:
#interviews
interviews = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/interviews.pkl')
interviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9642 entries, 0 to 9641
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     9642 non-null   object
 1   url      9642 non-null   object
 2   title    9642 non-null   object
 3   speaker  9642 non-null   object
 4   text     9642 non-null   object
dtypes: object(5)
memory usage: 376.8+ KB


In [3]:
interviews['speaker'] = interviews['speaker'].replace({'В.Путин': 'В.Путин:', 'Д.Медведев': 'Д.Медведев:'})

In [4]:
interviews.speaker.value_counts()

В.Путин:             4829
Д.Медведев:          1724
Вопрос:               458
А.Ванденко:           333
М.Келли:              156
                     ... 
Э.Хула                  1
Л.Фонтана               1
Дж.Стефанопулос         1
Дж.Миклетвейт           1
Газета «Политика»       1
Name: speaker, Length: 118, dtype: int64

In [5]:
filtered_speakers = interviews['speaker'].str.contains('В.Путин:|Д.Медведев')

In [6]:
filtered_interviews = interviews[(filtered_speakers)]

In [7]:
filtered_interviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6554 entries, 1 to 9641
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     6554 non-null   object
 1   url      6554 non-null   object
 2   title    6554 non-null   object
 3   speaker  6554 non-null   object
 4   text     6554 non-null   object
dtypes: object(5)
memory usage: 307.2+ KB


In [8]:
filtered_interviews.speaker.value_counts()

В.Путин:       4829
Д.Медведев:    1724
В.Путин: «        1
Name: speaker, dtype: int64

In [9]:
condition_1 = filtered_interviews['text'].str.contains('Опубликован в разделах:')
condition_2 = filtered_interviews['text'].str.contains('Дата публикации:')
condition_3 = filtered_interviews['text'].str.contains('Ссылка на материал:')
condition_4 = filtered_interviews['text'].str.strip().str.len() == 0
condition_5 = filtered_interviews['text'].str.contains('Официальный сайт президента России:')
condition_6 = filtered_interviews['text'].str.contains('http://')
condition_7 = filtered_interviews['text'].str.contains('<…>|\* \* \*')
condition_8 = filtered_interviews['text'].str.contains('Администрация Президента России')


filtered_interviews = filtered_interviews[~(condition_1 | condition_2 | condition_3| condition_4| condition_5| condition_6|condition_7|condition_8)]

In [11]:
def clean(text):
    # replace actual newline characters \n with a space
    text = re.sub(r'\n', ' ', text)
    #replace Д.Медведев:|В.Путин: with a white space
    text = re.sub(r'(Д\.Медведев:|В\.Путин:)', ' ', text)
    # replace everything non-alphanumeric with a space
    text = re.sub(r'\W+', ' ', text)
    # replace two or more dots with one
    text = re.sub(r'\.{2,}', ' ', text)
    # replace sequences of white spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # convert text to lowercase
    text = text.lower()
    # replace \xa0 with a white space
    text = re.sub(r'\xa0', ' ', text)
    return text.strip()

In [12]:
interviews['text_clean'] = interviews['text'].map(clean)

In [13]:
interviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9642 entries, 0 to 9641
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        9642 non-null   object
 1   url         9642 non-null   object
 2   title       9642 non-null   object
 3   speaker     9642 non-null   object
 4   text        9642 non-null   object
 5   text_clean  9642 non-null   object
dtypes: object(6)
memory usage: 452.1+ KB


In [14]:
interviews.to_pickle('data/corpus_adriana/corpus_president_of_russia/interviews.pkl')

In [10]:
def concat_strings(x):
    return ' '.join(x)

In [68]:
interviews_concatenated = filtered_interviews.groupby(['date', 'url', 'title', 'speaker'], as_index=False).agg({'text': concat_strings})

In [69]:
interviews_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     114 non-null    object
 1   url      114 non-null    object
 2   title    114 non-null    object
 3   speaker  114 non-null    object
 4   text     114 non-null    object
dtypes: object(5)
memory usage: 4.6+ KB


In [70]:
interviews_concatenated.text.iloc[0]

'Я не\xa0думаю, что открою для Вас какую‑то особую истину, если скажу, что работа у\xa0Президента очень интересная, но\xa0довольно сложная. И\xa0не\xa0важно, сколько тебе лет, 44\xa0года или 88,\xa0– это требует определённой мобилизации. Мы с\xa0Вами приблизительно одного возраста, поэтому Вы можете себе представить, какие эмоции в\xa0этом случае испытываешь. Хотя, конечно, с\xa0другой стороны, для того чтобы всё это понять и\xa0прочувствовать, нужно немножко посидеть в\xa0этом кресле. Это так, это правда. Что касается моей работы, то\xa0я могу сказать, что, наверное, те испытания, которые для меня являются значимыми, они сегодня очевидны. Это экономический кризис, который, к\xa0сожалению, случился в\xa0прошлом году. Я ещё только несколько месяцев был Президентом, когда в\xa0воздухе появился «аромат» кризиса, и\xa0все мы почувствовали, что это довольно серьёзно, что это рассчитано на\xa0значительное время. Другой очень большой, серьёзной проблемой были, конечно, те испытания, с\xa0кото

In [71]:
def clean(text):
    # replace actual newline characters \n with a space
    text = re.sub(r'\n', ' ', text)
    #replace Д.Медведев:|В.Путин: with a white space
    text = re.sub(r'(Д\.Медведев:|В\.Путин:)', ' ', text)
    # replace everything non-alphanumeric with a space
    text = re.sub(r'\W+', ' ', text)
    # replace two or more dots with one
    text = re.sub(r'\.{2,}', ' ', text)
    # replace sequences of white spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # convert text to lowercase
    text = text.lower()
    # replace \xa0 with a white space
    text = re.sub(r'\xa0', ' ', text)
    return text.strip()

In [72]:
interviews_concatenated['text_clean'] = interviews_concatenated['text'].map(clean)

In [73]:
interviews_concatenated

,date,url,title,speaker,text,text_clean
0,2009-09-20,http://kremlin.ru/events/president/transcripts...,Интервью телекомпании «Си-Эн-Эн» • Президент Р...,Д.Медведев:,"Я не думаю, что открою для Вас какую‑то особую...",я не думаю что открою для вас какую то особую ...
1,2009-11-07,http://kremlin.ru/events/president/transcripts...,Интервью немецкому журналу «Шпигель» • Президе...,Д.Медведев:,"Время бежит очень быстро. Спустя 20 лет всё, ч...",время бежит очень быстро спустя 20 лет всё что...
2,2010-02-25,http://kremlin.ru/events/president/transcripts...,Интервью французскому журналу «Пари-матч» • Пр...,Д.Медведев:,": Прежде всего хотел бы сказать, что мне очень...",прежде всего хотел бы сказать что мне очень пр...
3,2010-04-12,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева американскому телек...,Д.Медведев:,"То, что Россия – это нормальная современная ст...",то что россия это нормальная современная стран...
4,2010-04-14,http://kremlin.ru/events/president/transcripts...,Интервью журналисту «Известий» Мэлору Стуруа •...,Д.Медведев:,": Вы знаете, его легко оценивать. Это самый бе...",вы знаете его легко оценивать это самый бесспо...
...,...,...,...,...,...,...
109,2021-10-14,http://kremlin.ru/events/president/transcripts...,Интервью телеканалу CNBC • Президент России,В.Путин:,"Хочу вернуть вам эту шайбу в ответ. Нет, я на ...",хочу вернуть вам эту шайбу в ответ нет я на та...
110,2021-11-13,http://kremlin.ru/events/president/transcripts...,Интервью телеканалу «Россия» • Президент России,В.Путин:,"Во-первых, это вопрос не ко мне,\nпочему они н...",во первых это вопрос не ко мне почему они не р...
111,2022-02-03,http://kremlin.ru/events/president/transcripts...,Интервью Медиакорпорации Китая • Президент России,В.Путин:,: Мы давно знакомы\nс Председателем Си Цзиньпи...,мы давно знакомы с председателем си цзиньпином...
112,2022-06-03,http://kremlin.ru/events/president/transcripts...,Интервью телеканалу «Россия» • Президент России,В.Путин:,"Да, конечно, мы сейчас видим попытки переложит...",да конечно мы сейчас видим попытки переложить ...


In [74]:
interviews_concatenated.to_pickle('data/corpus_adriana/corpus_president_of_russia/interviews_concatenated.pkl')

### news

In [101]:
news = pd.read_excel('data/putin_corpus.xlsx')
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33189 entries, 0 to 33188
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          33189 non-null  object
 1   URL           33189 non-null  object
 2   description   30316 non-null  object
 3   introduction  30316 non-null  object
 4   id            33189 non-null  int64 
 5   speaker       33189 non-null  object
 6   p             33189 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.8+ MB


In [110]:
news.speaker.value_counts()

В.Путин                  20444
К.-Ж.Токаев                335
С.Абэ                      314
С.Брилёв                   271
А.Костин                   223
                         ...  
П.Тайченачев                 1
А.Марзуки                    1
С.Хасан                      1
Т.Николич (по-русски)        1
Йон Фредерик Баксаас         1
Name: speaker, Length: 944, dtype: int64

In [104]:
news.drop(['introduction','id'], axis=1, inplace=True)

In [105]:
news.rename(columns={'p':'text'},inplace=True)

In [111]:
filtered_speakers = news['speaker'].str.contains('В.Путин|Д.Медведев')

In [112]:
filtered_news = news[(filtered_speakers)]

In [113]:
filtered_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20585 entries, 0 to 33188
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         20585 non-null  object
 1   URL          20585 non-null  object
 2   description  18490 non-null  object
 3   speaker      20585 non-null  object
 4   text         20585 non-null  object
dtypes: object(5)
memory usage: 964.9+ KB


In [114]:
def concat_strings(x):
    return ' '.join(x)

In [115]:
news_concatenated = filtered_news.groupby(['date', 'URL', 'description', 'speaker'], as_index=False).agg({'text': concat_strings})

In [116]:
news_concatenated.text.iloc[0]

'Клянусь при осуществлении полномочий Президента Российской Федерации уважать и охранять права и свободы человека и гражданина, соблюдать и защищать Конституцию Российской Федерации, защищать суверенитет и независимость, безопасность и целостность государства, верно служить народу. Уважаемые граждане России! Дорогие друзья! Вступая в должность Президента Российской Федерации, понимаю всю свою ответственность перед Родиной. Её интересы, безопасность, благополучие граждан страны всегда были и всегда останутся для меня превыше всего. Сделаю всё, чтобы оправдать доверие миллионов наших граждан. Считаю смыслом всей своей жизни и своим долгом служение Отечеству, служение нашему народу, поддержка которого вдохновляет и помогает решать самые сложные и трудные задачи. Мы вместе прошли большой и сложный путь, поверили в себя, в свои силы, укрепили страну, вернули себе достоинство великой нации, мир увидел возрождённую Россию, и это результат усилий нашего народа, общей напряжённой работы, в кото

In [117]:
news_concatenated

,date,URL,description,speaker,text
0,2012-05-07,http://kremlin.ru/events/president/news/15224,В Большом Кремлёвском дворце состоялась торжес...,В.Путин,Клянусь при осуществлении полномочий Президент...
1,2012-05-07,http://kremlin.ru/events/president/news/15224,В Большом Кремлёвском дворце состоялась торжес...,Д.Медведев,Уважаемые граждане России! Дорогие друзья! Вст...
2,2012-05-07,http://kremlin.ru/events/president/news/15260,Глава государства приехал во Дворец спорта «Ме...,В.Путин,"Дорогие друзья, уважаемые любители спорта и хо..."
3,2012-05-09,http://kremlin.ru/events/president/news/15271,Владимир Путин присутствовал на военном параде...,В.Путин,Уважаемые граждане России! Дорогие ветераны! Т...
4,2012-05-09,http://kremlin.ru/events/president/news/15272,В Кремле состоялся торжественный приём в ознам...,В.Путин,"Дорогие наши ветераны! Уважаемые коллеги, друз..."
...,...,...,...,...,...
749,2022-06-30,http://kremlin.ru/events/president/news/68790,Глава государства возложил цветы к подножию мо...,В.Путин,Уважаемые товарищи! Дорогие ветераны! Мы собра...
750,2022-07-25,http://kremlin.ru/events/president/news/69059,Владимир Путин поздравил сотрудников органов с...,В.Путин,Уважаемые товарищи! Поздравляю вас с профессио...
751,2022-07-31,http://kremlin.ru/events/president/news/69082,В День Военно-Морского Флота Верховный Главнок...,В.Путин,"Товарищи матросы и старшины, мичманы и офицеры..."
752,2022-08-07,http://kremlin.ru/events/president/news/69123,Владимир Путин поздравил работников и ветерано...,В.Путин,Уважаемые друзья! Сегодня мы отмечаем День жел...


In [118]:
def clean(text):
    # replace actual newline characters \n with a space
    text = re.sub(r'\n', ' ', text)
    #replace Д.Медведев:|В.Путин: with a white space
    text = re.sub(r'(Д\.Медведев:|В\.Путин:)', ' ', text)
    # replace everything non-alphanumeric with a space
    text = re.sub(r'\W+', ' ', text)
    # replace two or more dots with one
    text = re.sub(r'\.{2,}', ' ', text)
    # replace sequences of white spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # convert text to lowercase
    text = text.lower()
    # replace \xa0 with a white space
    text = re.sub(r'\xa0', ' ', text)
    return text.strip()

In [119]:
news_concatenated['text_clean'] = news_concatenated['text'].map(clean)

In [120]:
news_concatenated

,date,URL,description,speaker,text,text_clean
0,2012-05-07,http://kremlin.ru/events/president/news/15224,В Большом Кремлёвском дворце состоялась торжес...,В.Путин,Клянусь при осуществлении полномочий Президент...,клянусь при осуществлении полномочий президент...
1,2012-05-07,http://kremlin.ru/events/president/news/15224,В Большом Кремлёвском дворце состоялась торжес...,Д.Медведев,Уважаемые граждане России! Дорогие друзья! Вст...,уважаемые граждане россии дорогие друзья вступ...
2,2012-05-07,http://kremlin.ru/events/president/news/15260,Глава государства приехал во Дворец спорта «Ме...,В.Путин,"Дорогие друзья, уважаемые любители спорта и хо...",дорогие друзья уважаемые любители спорта и хок...
3,2012-05-09,http://kremlin.ru/events/president/news/15271,Владимир Путин присутствовал на военном параде...,В.Путин,Уважаемые граждане России! Дорогие ветераны! Т...,уважаемые граждане россии дорогие ветераны тов...
4,2012-05-09,http://kremlin.ru/events/president/news/15272,В Кремле состоялся торжественный приём в ознам...,В.Путин,"Дорогие наши ветераны! Уважаемые коллеги, друз...",дорогие наши ветераны уважаемые коллеги друзья...
...,...,...,...,...,...,...
749,2022-06-30,http://kremlin.ru/events/president/news/68790,Глава государства возложил цветы к подножию мо...,В.Путин,Уважаемые товарищи! Дорогие ветераны! Мы собра...,уважаемые товарищи дорогие ветераны мы собрали...
750,2022-07-25,http://kremlin.ru/events/president/news/69059,Владимир Путин поздравил сотрудников органов с...,В.Путин,Уважаемые товарищи! Поздравляю вас с профессио...,уважаемые товарищи поздравляю вас с профессион...
751,2022-07-31,http://kremlin.ru/events/president/news/69082,В День Военно-Морского Флота Верховный Главнок...,В.Путин,"Товарищи матросы и старшины, мичманы и офицеры...",товарищи матросы и старшины мичманы и офицеры ...
752,2022-08-07,http://kremlin.ru/events/president/news/69123,Владимир Путин поздравил работников и ветерано...,В.Путин,Уважаемые друзья! Сегодня мы отмечаем День жел...,уважаемые друзья сегодня мы отмечаем день желе...


In [121]:
news_concatenated.to_pickle('data/corpus_adriana/corpus_president_of_russia/news_concatenated.pkl')